This notebook ingests NCA geotifs for the 'public website', applies colours to them based on examples provided alongside the data, and then generates static map tilesets, using the xyz tile schema. These tilesets are then uploaded to a dedicated S3 bucket.
Resources:
link to matplotlib colormaps - https://matplotlib.org/stable/gallery/color/colormap_reference.html

In [ ]:
import os
import numpy as np
import rasterio
from rasterio.plot import reshape_as_image
from matplotlib.colors import Normalize
from matplotlib.cm import get_cmap
import gdal2tiles
import folium

def generate_coloured_tif(data, colour_map_name, nodata):
    cmap = get_cmap(colour_map_name)

    na = data[~np.isnan(data)]
    min_value = min(na)
    max_value = max(na)

    norm = Normalize(vmin=min_value, vmax=max_value)

    coloured_data = (cmap(norm(data))[:, :, :3] * 255).astype(np.uint8)

    # Create an alpha band using the nodata value
    alpha_band = np.where(data == nodata, 0, 255).astype(np.uint8)

    # Stack the RGB and alpha bands
    coloured_data_with_alpha = np.dstack([coloured_data, alpha_band])

    return coloured_data_with_alpha

def process_geotiff(input_geotiff_path, output_directory, colour_map_name):
    try:
        with rasterio.open(input_geotiff_path) as src:
            # Load the data from the GeoTIFF
            tif_data = src.read(1, masked=True).astype('float32')
            nodata = src.nodata

            # Format the data (e.g., handle NaN values)
            tif_data_formatted = tif_data.filled(np.nan)

            # Generate the colored GeoTIFF image with alpha band
            coloured_image_with_alpha = generate_coloured_tif(tif_data_formatted, colour_map_name, nodata)

            # Define metadata for the colored GeoTIFF
            meta = src.meta.copy()
            meta.update({"count": 4, "dtype": 'uint8', "nodata": None})

            # Get the filename without extension
            file_name = os.path.splitext(os.path.basename(input_geotiff_path))[0]

            # Save the colored GeoTIFF locally
            coloured_geotiff_path = os.path.join(output_directory, f"{file_name}-coloured.tif")
            with rasterio.open(coloured_geotiff_path, 'w', **meta) as dst:
                # Write the 4-band data
                dst.write(coloured_image_with_alpha.transpose(2, 0, 1))

            return {
                'message': 'Colored GeoTIFF created successfully.',
                'coloured_geotiff_path': coloured_geotiff_path
            }
    except Exception as e:
        return {
            'error': str(e)
        }

def create_map_tiles(geotiff_path, output_directory):
    # Generate XYZ map tiles using gdal2tiles
    tiles_output_directory = os.path.join(output_directory, 'map_tiles')

    # Ensure the output directory exists
    os.makedirs(tiles_output_directory, exist_ok=True)

    # Generate tiles
    gdal2tiles.generate_tiles(
        geotiff_path,
        tiles_output_directory,
        zoom=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  # Adjust zoom levels as needed
        tmscompatible=True
    )

    return tiles_output_directory

# Usage example:
input_geotiff_path = 'data/Orana/Orana Rasters/groundcover/YQ-2022-GC10.tif'
output_directory = 'data/Orana/output_rasters/'
colour_map_name = 'viridis'


if 'coloured_geotiff_path' in processing_result:
    tiles_directory = create_map_tiles(processing_result['coloured_geotiff_path'], output_directory)

    # Display the map tiles using Folium
    m = folium.Map(location=[0, 0], zoom_start=1)
    tile_layer = folium.raster_layers.TileLayer(
        tiles=f'file://{tiles_directory}/{{z}}/{{x}}/{{y}}.png',
        attr='Custom Tiles',
        name='Map Tiles'
    )
    tile_layer.add_to(m)
    m.add_child(folium.LayerControl())
    display(m)
else:
    print("Error in processing GeoTIFF:", processing_result.get('error', 'Unknown Error'))

